<a href="https://colab.research.google.com/github/curiosity806/2020_dacon_satellite_precipitation/blob/master/pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
# install pytorch tpu version
# VERSION = "20200325"  #@param ["1.5" , "20200325", "nightly"]
# !curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
# !python pytorch-xla-env-setup.py --version $VERSION

In [0]:
import sys
import random
import os.path
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch_xla.core.xla_model as xm
import torch_xla.distributed.parallel_loader as pl
import torch_xla.distributed.xla_multiprocessing as xmp
from torchsummary import summary
from sklearn.metrics import f1_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [0]:
torch.__version__  # should be 1.5.0a0+d6149a7

'1.5.0a0+d6149a7'

## Load data

In [0]:
if not os.path.isfile('train.npy'):
    !cp '/content/drive/My Drive/2020 Kaggle Study/data/train.npy' train.npy
if not os.path.isfile('test.npy'):
    !cp '/content/drive/My Drive/2020 Kaggle Study/data/test.npy' test.npy
if not os.path.isfile('gmi_preci.npy'):
    !cp '/content/drive/My Drive/2020 Kaggle Study/data/gmi_preci/near2.npy' gmi_preci.npy

In [0]:
train = np.load('train.npy')  # float32
test = np.load('test.npy')  # float64
gmi_preci = np.load('gmi_preci.npy')

## Data preprocess

In [0]:
# train-valid split
train, valid, train_preci, valid_preci = train_test_split(train, gmi_preci, test_size=0.025, random_state=7777)

In [0]:
# 위경도 제거
train = np.concatenate((train[:,:,:,:10], train[:,:,:,-1:]), axis=3)
train.shape

(74436, 40, 40, 11)

In [0]:
# -9999를 포함한 이미지 제거
is_valid = (train[:,:,:,-1].reshape(-1, 1600) < 0).sum(axis=1) == 0
train = train[is_valid]
train_preci = train_preci[is_valid]

In [0]:
# fit scaler
scaler = StandardScaler()
scaler.fit(train[:,:,:,:9].reshape(-1, 9))

StandardScaler(copy=True, with_mean=True, with_std=True)

In [0]:
# 강수인 지역이 50픽셀 미만인 이미지 제거, land type 100 이상이 50픽셀 이상인 경우 살리기
is_valid = (train[:,:,:,-1].reshape(-1, 1600) >= 0.1).sum(axis=1) >= 50
is_valid = is_valid | ((train[:,:,:,9].reshape(-1, 1600) >= 100).sum(axis=1) >= 50)
train = train[is_valid]  # (-1, 40, 40, 15)
train_preci = train_preci[is_valid]

In [0]:
# gmi 강수량으로 대치
train[:, :, :, -1] = train_preci.reshape(-1, 40, 40)
# valid[:, :, :, -1] = valid_preci.reshape(-1, 40, 40)

In [0]:
# scale sensor values and land type
def scale(data):
    # normalize sensor data
    data[:,:,:,:9] = scaler.transform(data[:,:,:,:9].reshape(-1, 9)).reshape(-1, 40, 40, 9)

    # ocean 0.0, land 1.0, coastal 0.3, inland water 0.7
    land_type_data = data[:,:,:,9]
    data[:,:,:,9] = np.where(land_type_data//100 == 2, 0.3,
                             np.where(land_type_data//100 == 3, 0.7,
                                      land_type_data//100))
    return data

train = scale(train)
valid = scale(valid)
test = scale(test)

In [0]:
# pytorch is channel first
train = train.transpose((0, 3, 1, 2))
valid = valid.transpose((0, 3, 1, 2))
test = test.transpose((0, 3, 1, 2))
train.shape, valid.shape, test.shape

((51073, 11, 40, 40), (1909, 15, 40, 40), (2416, 14, 40, 40))

## Dataset

In [0]:
def r(a, n=1):
    return np.rot90(a, n, (1, 2))
def f_ud(a):
    return np.flip(a, 1)
def f_lr(a):
    return np.flip(a, 2)

In [0]:
tf_list = [lambda x: x,              # abcd
           lambda x: r(f_lr(x)),     # acbd
           lambda x: f_lr(x),        # badc
           lambda x: r(x),           # bdac
           lambda x: r(x, 3),        # cadb
           lambda x: f_ud(x),        # cdab
           lambda x: f_ud(r(x, 3)),  # dbca
           lambda x: r(x, 2)]        # dcba

In [0]:
class MapDataset(torch.utils.data.Dataset):
    def __init__(self, data, is_train, transform=None):
        self.data = data
        self.is_train = is_train
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        if self.transform:
            rand_num = random.randint(0, 7)
            sample = self.transform[rand_num](self.data[idx])
        else:
            sample = self.data[idx]

        sample = torch.from_numpy(sample.copy())

        if self.is_train:
            return sample[:10], sample[-1:]
        else:
            return sample[:10]

## Define metrics

In [0]:
def mae(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    y_true = y_true.reshape(1, -1)[0]
    y_pred = y_pred.reshape(1, -1)[0]
    over_threshold = y_true >= 0.1
    return np.mean(np.abs(y_true[over_threshold] - y_pred[over_threshold]))

def fscore(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    y_true = y_true.reshape(1, -1)[0]
    y_pred = y_pred.reshape(1, -1)[0]
    remove_NAs = y_true >= 0
    y_true = np.where(y_true[remove_NAs] >= 0.1, 1, 0)
    y_pred = np.where(y_pred[remove_NAs] >= 0.1, 1, 0)
    return (f1_score(y_true, y_pred))

def maeOverFscore(y_true, y_pred):
    return mae(y_true, y_pred) / (fscore(y_true, y_pred) + 1e-07)

## Model

In [0]:
class DoubleConv(nn.Module):
    """(convolution => [BN] => ReLU) * 2"""

    def __init__(self, in_channels, out_channels, mid_channels=None):
        super().__init__()
        if not mid_channels:
            mid_channels = out_channels
        self.double_conv = nn.Sequential(
            nn.Conv2d(in_channels, mid_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(mid_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(mid_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.double_conv(x)


class Down(nn.Module):
    """Downscaling with maxpool then double conv"""

    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.maxpool_conv = nn.Sequential(
            nn.MaxPool2d(2),
            DoubleConv(in_channels, out_channels)
        )

    def forward(self, x):
        return self.maxpool_conv(x)


class Up(nn.Module):
    """Upscaling then double conv"""

    def __init__(self, in_channels, out_channels, bilinear=True):
        super().__init__()

        # if bilinear, use the normal convolutions to reduce the number of channels
        if bilinear:
            self.up = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
            self.conv = DoubleConv(in_channels, out_channels, in_channels // 2)
        else:
            self.up = nn.ConvTranspose2d(in_channels , in_channels // 2, kernel_size=2, stride=2)
            self.conv = DoubleConv(in_channels, out_channels)


    def forward(self, x1, x2):
        x1 = self.up(x1)
        # input is CHW
        diffY = x2.size()[2] - x1.size()[2]
        diffX = x2.size()[3] - x1.size()[3]

        x1 = F.pad(x1, [diffX // 2, diffX - diffX // 2,
                        diffY // 2, diffY - diffY // 2])
        # if you have padding issues, see
        # https://github.com/HaiyongJiang/U-Net-Pytorch-Unstructured-Buggy/commit/0e854509c2cea854e247a9c615f175f76fbb2e3a
        # https://github.com/xiaopeng-liao/Pytorch-UNet/commit/8ebac70e633bac59fc22bb5195e513d5832fb3bd
        x = torch.cat([x2, x1], dim=1)
        return self.conv(x)


class OutConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(OutConv, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=1)

    def forward(self, x):
        return self.conv(x)

In [0]:
class UNet(nn.Module):
    def __init__(self, n_channels, n_classes, bilinear=True):
        super(UNet, self).__init__()
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.bilinear = bilinear

        self.inc = DoubleConv(n_channels, 64)
        self.down1 = Down(64, 128)
        self.down2 = Down(128, 256)
        self.down3 = Down(256, 512)
        factor = 2 if bilinear else 1
        self.down4 = Down(512, 1024 // factor)
        self.up1 = Up(1024, 512 // factor, bilinear)
        self.up2 = Up(768, 256 // factor, bilinear)
        self.up3 = Up(256, 128 // factor, bilinear)
        self.up4 = Up(128, 64, bilinear)
        self.outc = OutConv(64, n_classes)

    def forward(self, x):
        x1 = self.inc(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        # x5 = self.down4(x4)
        # x = self.up1(x5, x4)
        # x = self.up2(x, x3)
        x = self.up2(x4, x3)
        x = self.up3(x, x2)
        x = self.up4(x, x1)
        logits = self.outc(x)
        return logits

In [0]:
summary(UNet(10, 1), (10, 40, 40))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 40, 40]           5,824
       BatchNorm2d-2           [-1, 64, 40, 40]             128
              ReLU-3           [-1, 64, 40, 40]               0
            Conv2d-4           [-1, 64, 40, 40]          36,928
       BatchNorm2d-5           [-1, 64, 40, 40]             128
              ReLU-6           [-1, 64, 40, 40]               0
        DoubleConv-7           [-1, 64, 40, 40]               0
         MaxPool2d-8           [-1, 64, 20, 20]               0
            Conv2d-9          [-1, 128, 20, 20]          73,856
      BatchNorm2d-10          [-1, 128, 20, 20]             256
             ReLU-11          [-1, 128, 20, 20]               0
           Conv2d-12          [-1, 128, 20, 20]         147,584
      BatchNorm2d-13          [-1, 128, 20, 20]             256
             ReLU-14          [-1, 128,

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:2875: UserWarning: The default behavior for interpolate/upsample with float scale_factor will change in 1.6.0 to align with other frameworks/libraries, and use scale_factor directly, instead of relying on the computed output size. If you wish to keep the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/upsample with float scale_factor will change "


## 강수량 보정

In [0]:
dr = [(-1, -1), (-1, 0), (-1, 1),
      (0, -1), (0, 0), (0, 1),
      (1, -1), (1, 0), (1, 1)]

def get_dist(p1, p2):  # p1, p2: shape=(-1, 2).
    x1, y1 = np.deg2rad(p1[:,0]), np.deg2rad(p1[:,1])
    x2, y2 = np.deg2rad(p2[:,0]), np.deg2rad(p2[:,1])
    dlon = x2 - x1
    dlat = y2 - y1
    a = np.sin(dlat/2)**2 + np.cos(y1) * np.cos(y2) * np.sin(dlon/2)**2 
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))    
    return 6373.0 * c  # km, shape=(-1).

# value: (40, 40)
# ori_ll, tgt_ll: (40, 40, 2)
def compen_ll(value, ori_ll, tgt_ll):  # ori_ll에서의 value를 tgt_ll에 대한 값으로 바꿈
    ret = np.empty_like(value)
    n, m = value.shape[0], value.shape[1]
    for i in range(n):
        for j in range(m):
            nears = []  # (row, col, value)
            for k in range(9):
                ii = i + dr[k][0]
                jj = j + dr[k][1]
                if ii >= 0 and ii < n and jj >= 0 and jj < m:
                    nears.append((ori_ll[ii, jj][0], ori_ll[ii, jj][1],
                                  tgt_ll[i, j][0], tgt_ll[i, j][1],
                                  value[ii, jj]))
            nears = np.array(nears)  # shape=(-1, 5)
            dists = get_dist(nears[:, 0:2], nears[:, 2:4]).reshape(-1, 1)
            values = nears[:, 4].reshape(-1, 1)
            nears = np.concatenate((dists, values), 1)
            nears = nears[np.argsort(nears[:, 0])]  # sort by dist
            nears = nears[:2, :]  # 가까운 점 2개만 고려

            weights = 1 / (nears[:, 0] ** 2 + sys.float_info.epsilon)
            weighted_sum = (weights * nears[:, 1]).sum()
            ret[i, j] = weighted_sum / weights.sum()
    return ret

In [0]:
from multiprocessing import Process, Manager

def proc_func(splitted, dpr_preci, proc_id):
    part = splitted[proc_id]
    arr = np.empty_like(part[:, :, :, 14])  # shape=(-1, 40, 40)
    for i in range(part.shape[0]):
        arr[i, :, :] = compen_ll(part[i, :, :, 14], part[i, :, :, 10:12], part[i, :, :, 12:14])
    dpr_preci[proc_id] = arr

def gmi2dpr(test, gmi_preci):
    n_procs = 4
    procs = []
    manager = Manager()
    dpr_preci = manager.list([None] * n_procs)

    data = np.concatenate((test, gmi_preci), axis=3)
    n_imgs = data.shape[0]  # split data into n_procs arrays
    splitted = np.split(data, np.arange(n_imgs // n_procs + n_imgs % n_procs, n_imgs, n_imgs // n_procs))

    for proc_id in range(n_procs):
        proc = Process(target=proc_func, args=(splitted, dpr_preci, proc_id))
        proc.start()
        procs.append(proc)

    for proc in procs:
        proc.join()

    dpr_preci = np.concatenate(dpr_preci)
    return dpr_preci

## TPU training

In [0]:
def map_fn(index, flags):
    device = xm.xla_device()  # TPU

    train_dataset = MapDataset(train, True, tf_list)
    valid_dataset = MapDataset(valid, False)
    test_dataset = MapDataset(test, False)

    train_sampler = torch.utils.data.distributed.DistributedSampler(
        train_dataset,
        num_replicas=xm.xrt_world_size(),
        rank=xm.get_ordinal(),
        shuffle=True)

    train_loader = torch.utils.data.DataLoader(train_dataset,
                                               batch_size=flags['batch_size'],
                                               sampler=train_sampler,
                                               num_workers=flags['num_workers'],
                                               drop_last=True)  # small batch is not good for bn
    valid_loader = torch.utils.data.DataLoader(valid_dataset,
                                               batch_size=len(valid),
                                               num_workers=1,
                                               drop_last=False)
    test_loader = torch.utils.data.DataLoader(test_dataset,
                                              batch_size=len(test),
                                              num_workers=1,
                                              drop_last=False)

    # min_score 초기화
    min_score = 2.0
    cnt = 0

    # net, loss_fn, optimizer 생성
    net = UNet(10, 1).to(device).train()  # Training mode
    loss_fn = torch.nn.L1Loss()  # MAE
    optimizer = torch.optim.Adam(net.parameters(), lr=0.001)

    # Training
    for epoch in range(flags['num_epochs']):
        # Training mode
        net.train()

        # 병렬 연산을 위한 loader
        para_train_loader = pl.ParallelLoader(train_loader, [device]).per_device_loader(device)

        # Update parameters
        for batch_num, batch in enumerate(para_train_loader):
            data, targets = batch

            # Inference -> loss 계산 -> gradient 초기화 -> backpropagation
            output = net(data)  # Acquires the network's best guesses
            loss = loss_fn(output, targets)  # Computes loss
            optimizer.zero_grad()
            loss.backward()  # Updates model

            xm.optimizer_step(optimizer)  # Note: barrier=True not needed when using ParallelLoader

        # Boosting
        

        # Main core에서만 실행
        if xm.is_master_ordinal():
            print('epoch', epoch, end=': ')

            # Evaluation mode
            net.eval()

            # Prediction
            with torch.no_grad():
                for batch_num, batch in enumerate(valid_loader):
                    output = net(batch.to(device))  # (-1, 1, 40, 40)

            # Numpy로 변환
            output = output.cpu().numpy()

            # Score 계산
            mae_score = mae(valid_preci, output)
            f_score = fscore(valid_preci, output)
            mae_over_fscore = maeOverFscore(valid_preci, output)
            print(f'mae={mae_score}, fscore={f_score}, mae/fscore={mae_over_fscore}')

            # Best model 저장
            if mae_over_fscore < min_score:
                min_score = mae_over_fscore
                torch.save(net.state_dict(), 'best_model.pt')
                print('model saved.')
                cnt = 0
            else:
                cnt += 1

        xm.rendezvous('validation')

        if cnt >= 10:
            break

    # Main core에서만 실행
    if xm.is_master_ordinal():
        # Best model 불러오기
        net.load_state_dict(torch.load('best_model.pt'))
        print('model loaded.')

        # Evaluation mode로 전환
        net.eval()

        # Final validation
        with torch.no_grad():
            for batch_num, batch in enumerate(valid_loader):
                output = net(batch.to(device))  # (-1, 1, 40, 40)

        # Numpy로 변환
        output = output.cpu().numpy()

        # DPR 강수량으로 보정
        output = gmi2dpr(valid.transpose((0, 2, 3, 1))[:, :, :, :-1], output.transpose((0, 2, 3, 1)))

        # Score 계산
        mae_score = mae(valid[:,-1:,:,:], output)
        f_score = fscore(valid[:,-1:,:,:], output)
        mae_over_fscore = maeOverFscore(valid[:,-1:,:,:], output)
        print(f'final val: mae={mae_score}, fscore={f_score}, mae/fscore={mae_over_fscore}')

        # Prediction
        with torch.no_grad():
            for batch_num, batch in enumerate(test_loader):
                output = net(batch.to(device))  # (-1, 1, 40, 40)

        # Numpy로 변환
        output = output.cpu().numpy()

        # 강수량 보정
        output = gmi2dpr(test.transpose((0, 2, 3, 1)), output.transpose((0, 2, 3, 1)))

        # Submission file 올리기
        submission = pd.read_csv('/content/drive/My Drive/2020 Kaggle Study/data/sample_submission.csv')
        submission.iloc[:,1:] = output.reshape(-1, 1600)
        submission.to_csv('/content/drive/My Drive/2020 Kaggle Study/submission/unet_compen.csv', index=False)
        print('submission uploaded.')

In [0]:
# Configures training (and evaluation) parameters
flags = {'batch_size': 128,
         'num_workers': 8,
         'num_epochs': 150,
         'seed': 7777}

xmp.spawn(map_fn, args=(flags,), nprocs=8, start_method='fork')

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:2875: UserWarning: The default behavior for interpolate/upsample with float scale_factor will change in 1.6.0 to align with other frameworks/libraries, and use scale_factor directly, instead of relying on the computed output size. If you wish to keep the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/upsample with float scale_factor will change "
/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:2875: UserWarning: The default behavior for interpolate/upsample with float scale_factor will change in 1.6.0 to align with other frameworks/libraries, and use scale_factor directly, instead of relying on the computed output size. If you wish to keep the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/u

epoch 0: mae=1.5476971864700317, fscore=0.5676967247176156, mae/fscore=2.726274164453693
epoch 1: mae=1.4379225969314575, fscore=0.6458771579293932, mae/fscore=2.2263093788768304
epoch 2: mae=1.4013592004776, fscore=0.6681043856469735, mae/fscore=2.097515030333262
epoch 3: mae=1.320988655090332, fscore=0.6946790966916838, mae/fscore=1.9015808467870678
model saved.
epoch 4: mae=1.3048819303512573, fscore=0.7037970975577009, mae/fscore=1.8540595712506747
model saved.
epoch 5: mae=1.2572697401046753, fscore=0.7009101885390545, mae/fscore=1.793766992242718
model saved.
epoch 6: mae=1.2396239042282104, fscore=0.7210986398796115, mae/fscore=1.719076508766563
model saved.
epoch 7: mae=1.2726143598556519, fscore=0.7109905083670098, mae/fscore=1.789917257526888
epoch 8: mae=1.1979784965515137, fscore=0.7281561955472788, mae/fscore=1.6452216424924524
model saved.
epoch 9: mae=1.208752989768982, fscore=0.730284617538452, mae/fscore=1.6551804532392356
epoch 10: mae=1.1988269090652466, fscore=0.735